# Init values

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
SEED = 42
# dataset_folder = "datasets/financial"
dataset_folder = "datasets/healthcare"
# dataset_folder = "datasets/eligibility"

MAX_ITERATIONS = 1
TEMPERATURE = 0.9
LLM_NAME = "meta-llama/llama-3.1-8b-instruct:free"
# LLM_NAME = "meta-llama/llama-3.1-8b-instruct:free"


In [3]:
from dotenv import load_dotenv
from langchain_community.cache import SQLiteCache
from langchain.globals import set_llm_cache
from caia.utils import save_yaml_results
from caia.utils import ChatOpenRouter

import json
import uuid


# dataset_folder = "datasets/financial"
with open(f'{dataset_folder}/dataset_description.json', 'r') as f:
    dataset_description = json.load(f)

load_dotenv("env")
# set_llm_cache(SQLiteCache(database_path=".cache_langchain.db"))

llm_generator = ChatOpenRouter(model_name=LLM_NAME, cache=False,
                               temperature=TEMPERATURE) 

In [4]:
from rich import print

training_code = """
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# load the old data
dataset_folder = "datasets/healthcare"
X_train_old = pd.read_csv(f"{dataset_folder}/X_train_old.csv")
X_test_old = pd.read_csv(f"{dataset_folder}/X_test_old.csv")
y_train_old = pd.read_csv(f"{dataset_folder}/y_train_old.csv").squeeze("columns")
y_test_old = pd.read_csv(f"{dataset_folder}/y_test_old.csv").squeeze("columns")

model_old = RandomForestClassifier(random_state=42)


model_old.fit(X_train_old, y_train_old)

# Test the model on the old test set
old_accuracy = model_old.score(X_test_old, y_test_old)

print(f'Model trained and evaluated on the old distribution: {old_accuracy}')
"""
print(training_code)

import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# load the old data
dataset_folder = "datasets/healthcare"
X_train_old = pd.read_csv(f"{dataset_folder}/X_train_old.csv")
X_test_old = pd.read_csv(f"{dataset_folder}/X_test_old.csv")
y_train_old = pd.read_csv(f"{dataset_folder}/y_train_old.csv").squeeze("columns")
y_test_old = pd.read_csv(f"{dataset_folder}/y_test_old.csv").squeeze("columns")

model_old = RandomForestClassifier(random_state=42)


model_old.fit(X_train_old, y_train_old)

# Test the model on the old test set
old_accuracy = model_old.score(X_test_old, y_test_old)

print(f'Model trained and evaluated on the old distribution: {old_accuracy}')

In [5]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier


# load the reference data
X_train_old = pd.read_csv(f"{dataset_folder}/X_train_old.csv")
X_test_old = pd.read_csv(f"{dataset_folder}/X_test_old.csv")
y_train_old = pd.read_csv(f"{dataset_folder}/y_train_old.csv").squeeze("columns")
y_test_old = pd.read_csv(f"{dataset_folder}/y_test_old.csv").squeeze("columns")

print(f"X_train_old shape: {X_train_old.shape}")
print(f"X_test_old shape: {X_test_old.shape}")

model_old = RandomForestClassifier(random_state=SEED)
model_old.fit(X_train_old, y_train_old)

# Test the model on the initial test set
initial_accuracy = model_old.score(X_test_old, y_test_old)

print(f'Model trained and evaluated on the old distribution: {initial_accuracy}')

# Test the model on the drifted test set
X_test_new = pd.read_csv(f"{dataset_folder}/X_test_new.csv")
y_test_new = pd.read_csv(f"{dataset_folder}/y_test_new.csv").squeeze("columns")

print(f"X_test_new shape: {X_test_new.shape}")
drifted_accuracy = model_old.score(X_test_new, y_test_new)
print(f'Model evaluated on the new distribution: {drifted_accuracy}')

# calcualte the average accuracy
average_accuracy = (initial_accuracy + drifted_accuracy) / 2
print(f'Average accuracy on both distributions: {average_accuracy}')

X_train_old shape: (700, 10)

X_test_old shape: (300, 10)

Model trained and evaluated on the old distribution: 0.86

X_test_new shape: (30, 10)

Model evaluated on the new distribution: 0.7333333333333333

Average accuracy on both distributions: 0.7966666666666666

In [6]:
metrics = {"model_old_score": {
            "on_new_data": initial_accuracy,
            "on_old_data": drifted_accuracy
        }
}
metrics

{'model_old_score': {'on_new_data': 0.86, 'on_old_data': 0.7333333333333333}}

# Baseline agent

In [7]:
from caia.benchmark.baseline import StandardGraph
standard_graph = StandardGraph(llm_generator, debug=False)

initial_state = {
    "model_code": training_code,
    "metrics":metrics,
    "max_iterations": MAX_ITERATIONS,
    "dataset_description": dataset_description
}

output = standard_graph.run(initial_state)

short_uuid = str(uuid.uuid4())[:8]
filename = f"results/baseline_temp_{TEMPERATURE}_max_iter_{MAX_ITERATIONS}_llm_{LLM_NAME.split('/')[1].split(':')[0]}_dataset_{dataset_folder.split('/')[-1]}_{short_uuid}.yaml"
save_yaml_results(output, filename)

Max iterations set to: 1

🚀 Starting Improved Baseline Model Improvement Process

Dataset: Chronic Condition Prediction Data

Features: 10 total, 6 numerical, 4 categorical

Executing Node: improve_code

...

Initial metrics: {'old_distribution': 0.7333333333333333, 'new_distribution': 0.86}

Final metrics: {}

Improvement path: 0 entries

Total tokens used: 1699

Results saved to: results/baseline_temp_0.9_max_iter_1_llm_llama-3.1-8b-instruct_dataset_healthcare_7fa34f40.yaml

# ReAct agent

In [ ]:
from caia.benchmark.react import ReactImprover

# Initialize the React improver with your LLM
react_graph = ReactImprover(llm_generator)

# Prepare initial state
initial_state = {
    "model_code": training_code,
    "metrics": metrics,
    "max_iterations": MAX_ITERATIONS,
    "dataset_description": dataset_description
}

# Run the agent
output = react_graph.run(initial_state)

short_uuid = str(uuid.uuid4())[:8]
filename = f"results/react_temp_{TEMPERATURE}_max_iter_{MAX_ITERATIONS}_llm_{LLM_NAME.split('/')[1].split(':')[0]}_dataset_{dataset_folder.split('/')[-1]}_{short_uuid}.yaml"
save_yaml_results(output, filename)

Max iterations set to: 1

🚀 Starting React-based Model Improvement Process

Dataset: Chronic Condition Prediction Data

Error handling: stopping after 3 consecutive failures

Features: 10 total, 6 numerical, 4 categorical

...

New Distribution: +0.0000

Distribution Gap: 0.1033 (changed by +0.0000)

Iteration Times:

Iteration 1: 28.58 seconds

Results saved to: results/react_temp_1.0_max_iter_1_llm_llama-3.1-8b-instruct_dataset_healthcare_6fd26f64.yaml

# Plan and execute agent

In [14]:
from caia.benchmark.plan_and_execute import PlanAndExecuteGraph

# Initialize with max_failures parameter
plan_execute_graph = PlanAndExecuteGraph(
    llm_generator, 
    max_iterations=MAX_ITERATIONS,
    max_failures=3  # Will stop after 3 consecutive failures
)

# Prepare initial state
initial_state = {
    "model_code": training_code,
    "metrics": metrics,
    "dataset_description": dataset_description
}

# Run the agent
output = plan_execute_graph.run(initial_state)

# create a short version of uuid using python


short_uuid = str(uuid.uuid4())[:8]
filename = f"results/planexecute_temp_{TEMPERATURE}_max_iter_{MAX_ITERATIONS}_llm_{LLM_NAME.split('/')[1].split(':')[0]}_dataset_{dataset_folder.split('/')[-1]}_{short_uuid}.yaml"


save_yaml_results(output, filename)

🚀 Starting Plan-and-Execute Model Improvement Process

Dataset: Chronic Condition Prediction Data

Error handling: stopping after 3 consecutive failures

Features: 10 total, 6 numerical, 4 categorical

🧠 PLANNING STEP

...

Initial metrics: {'old_distribution': 0.7333333333333333, 'new_distribution': 0.86}

Final metrics: {'old_distribution': 0.8766666666666667, 'new_distribution': 0.8333333333333334}

Improvement path: 1 entries

Total tokens used: 8027

Results saved to: 
results/planexecute_temp_1.0_max_iter_1_llm_llama-3.1-8b-instruct_dataset_healthcare_6e8c2b15.yaml

# Reflection agent

In [15]:
from caia.benchmark.reflection import ReflectionGraph


# Initialize with both max_iterations and max_failures parameters
reflection_graph = ReflectionGraph(
    llm_generator, 
    max_iterations=MAX_ITERATIONS,
    max_failures=3  # Will stop after 3 consecutive failures
)

# Prepare initial state
initial_state = {
    "model_code": training_code,
    "metrics": metrics,
    "dataset_description": dataset_description
}

# Run the agent
output = reflection_graph.run(initial_state)

short_uuid = str(uuid.uuid4())[:8]
filename = f"results/reflection_temp_{TEMPERATURE}_max_iter_{MAX_ITERATIONS}_llm_{LLM_NAME.split('/')[1].split(':')[0]}_dataset_{dataset_folder.split('/')[-1]}_{short_uuid}.yaml"
save_yaml_results(output, filename)

🚀 Starting Reflection-Based Model Improvement Process

Dataset: Chronic Condition Prediction Data

Error handling: stopping after 3 consecutive failures

Features: 10 total, 6 numerical, 4 categorical

🔍 GENERATING IMPROVED CODE

...

Final metrics: {'old_distribution': 0.8766666666666667, 'new_distribution': 0.8333333333333334}

Improvement path: 1 entries

Reflections: 1

Total tokens used: 6421

Results saved to: results/reflection_temp_1.0_max_iter_1_llm_llama-3.1-8b-instruct_dataset_healthcare_42cb6f9e.yaml

# ToT

In [17]:
from caia.benchmark.tot import TreeOfThoughtsGraph
import json



# Initialize with ToT-specific parameters
tot_graph = TreeOfThoughtsGraph(
    llm_generator, 
    max_iterations=MAX_ITERATIONS,
    beam_width=3,           # Number of candidates to keep after pruning
    num_candidates=3,       # Number of candidates to generate in each expansion
    threshold=0.9,          # Score threshold for accepting a solution
    max_depth=3,            # Maximum search depth in the tree
    max_failures=3          # Will stop after 3 consecutive failures
)

# Prepare initial state
initial_state = {
    "model_code": training_code,
    "metrics": metrics,
    "dataset_description": dataset_description
}

# Run the agent
output = tot_graph.run(initial_state)

short_uuid = str(uuid.uuid4())[:8]
filename = f"results/tot_temp_{TEMPERATURE}_max_iter_{MAX_ITERATIONS}_llm_{LLM_NAME.split('/')[1].split(':')[0]}_dataset_{dataset_folder.split('/')[-1]}_{short_uuid}.yaml"
save_yaml_results(output, filename)

🚀 Starting Tree of Thoughts Model Improvement Process

Parameters: max_iterations=1, beam_width=3, num_candidates=3

Error handling: stopping after 3 consecutive failures

Dataset: Chronic Condition Prediction Data

Features: 10 total, 6 numerical, 4 categorical

...

Final metrics: {'old_distribution': 0.8633333333333333, 'new_distribution': 0.8333333333333334}

Improvement path: 1 entries

Total tokens used: 2852

Execution stats: 1 successes, 0 failures

Results saved to: results/tot_temp_1.0_max_iter_1_llm_llama-3.1-8b-instruct_dataset_healthcare_753ac1fe.yaml

# Self-Discovery

In [22]:
from caia.benchmark.self_discover import SelfDiscoverGraph

# Initialize with both max_iterations and max_failures
self_discovery_agent = SelfDiscoverGraph(
    llm_generator, 
    max_iterations=MAX_ITERATIONS,
    max_failures=4  # Will stop after 3 consecutive failures
)

# Prepare initial state
initial_state = {
    "model_code": training_code,
    "metrics": metrics,
    "dataset_description": dataset_description
}

# Run the agent
output = self_discovery_agent.run(initial_state)

short_uuid = str(uuid.uuid4())[:8]
filename = f"results/selfdiscovery_temp_{TEMPERATURE}_max_iter_{MAX_ITERATIONS}_llm_{LLM_NAME.split('/')[1].split(':')[0]}_dataset_{dataset_folder.split('/')[-1]}_{short_uuid}.yaml"
save_yaml_results(output, filename)


🚀 Starting Self-Discovery Model Improvement Process
Dataset: Chronic Condition Prediction Data
Error handling: stopping after 4 consecutive failures
Features: 10 total, 6 numerical, 4 categorical

🔍 SELECTING REASONING MODULES
Selected modules: 3
- 1. How could I simpl...
- 2. What are the key ...
- 3. How can I impleme...

Current Token Usage:
Prompt: 0
Completion: 0
Total: 0

🛠️ ADAPTING MODULES
Adapted modules: **Adaptation of Selected Modules**

**1. Simplify the Problem**

To simplify the problem, we can focus on the most relevant features that contribute to the outcome. We can:

* Select the top 3 numerical features (Age, BMI, Blood Pressure) and the top 2 categorical features (Smoking Status, Diet Quality) based on correlation analysis or feature importance.
* Consider a binary classification problem (e.g., healthy vs. unhealthy) to start with, and later expand to a multi-class classification problem (e.g., different health conditions).

**2. Improve Model on Distribution Shift

Results saved to: 
results/selfdiscovery_temp_1.0_max_iter_1_llm_llama-3.1-8b-instruct_dataset_healthcare_a2a7ee19.yaml

# Improver

## Semantic memory

### Training code

In [25]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# load the old data
dataset_folder = "datasets/healthcare"
X_train_old = pd.read_csv(f"{dataset_folder}/X_train_old.csv")
X_test_old = pd.read_csv(f"{dataset_folder}/X_test_old.csv")
y_train_old = pd.read_csv(f"{dataset_folder}/y_train_old.csv").squeeze("columns")
y_test_old = pd.read_csv(f"{dataset_folder}/y_test_old.csv").squeeze("columns")

model_old = RandomForestClassifier(random_state=42)


model_old.fit(X_train_old, y_train_old)

# Test the model on the old test set
old_accuracy = model_old.score(X_test_old, y_test_old)

print(f'Model trained and evaluated on the old distribution: {old_accuracy}')

Model trained and evaluated on the old distribution: 0.86

In [26]:
from caia.memory import WorkingMemory, EpisodicMemory, SemanticMemory
from caia.memory import Dataset


# tools = get_tools([calculate_trust_score])


# At the beginning, the agent has 1 entry in the semantic memory. 
# Here we put the path of each dataset file in the semantic memory.
dataset_old = Dataset(X_train=f"{dataset_folder}/X_train_old.csv",
                                     X_test=f"{dataset_folder}/X_test_old.csv",
                                     y_train=f"{dataset_folder}/y_train_old.csv",
                                     y_test=f"{dataset_folder}/y_test_old.csv",
                                     description=dataset_description)

model_code = """
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# load the old data
dataset_folder = "datasets/healthcare"
X_train_old = pd.read_csv(f"{dataset_folder}/X_train_old.csv")
X_test_old = pd.read_csv(f"{dataset_folder}/X_test_old.csv")
y_train_old = pd.read_csv(f"{dataset_folder}/y_train_old.csv").squeeze("columns")
y_test_old = pd.read_csv(f"{dataset_folder}/y_test_old.csv").squeeze("columns")

model_old = RandomForestClassifier(random_state=42)


model_old.fit(X_train_old, y_train_old)

# Test the model on the old test set
old_accuracy = model_old.score(X_test_old, y_test_old)

print(f'Model trained and evaluated on the old distribution: {old_accuracy}')
"""

init_semantic_memory = SemanticMemory(dataset_old=dataset_old, 
                                        model_object=model_old, 
                                        model_code=model_code)
# semantic_memory
print(init_semantic_memory.model_code)

import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# load the old data
dataset_folder = "datasets/healthcare"
X_train_old = pd.read_csv(f"{dataset_folder}/X_train_old.csv")
X_test_old = pd.read_csv(f"{dataset_folder}/X_test_old.csv")
y_train_old = pd.read_csv(f"{dataset_folder}/y_train_old.csv").squeeze("columns")
y_test_old = pd.read_csv(f"{dataset_folder}/y_test_old.csv").squeeze("columns")

model_old = RandomForestClassifier(random_state=42)


model_old.fit(X_train_old, y_train_old)

# Test the model on the old test set
old_accuracy = model_old.score(X_test_old, y_test_old)

print(f'Model trained and evaluated on the old distribution: {old_accuracy}')

## Episodic memory

In [27]:
from caia.memory import Dataset
from docarray import DocList

dataset_new = Dataset(X_train=f"{dataset_folder}/X_train_new.csv",
                        X_test=f"{dataset_folder}/X_test_new.csv",
                        y_train=f"{dataset_folder}/y_train_new.csv",
                        y_test=f"{dataset_folder}/y_test_new.csv",
                        description=dataset_description)


first_episodic_memory = EpisodicMemory(dataset_new=dataset_new,
                                        quick_insight={},
                                       deep_insight=None)
init_episodic_memory = DocList[EpisodicMemory]([first_episodic_memory])
init_episodic_memory[0]


📄 EpisodicMemory : 757399f ...
╭────────────────────────────┬─────────╮
│ Attribute                  │ Value   │
├────────────────────────────┼─────────┤
│ quick_insight: dict        │ {}      │
╰────────────────────────────┴─────────╯
└── 🔶 dataset_new: Dataset
    └── 📄 Dataset : a26cb97 ...
        ╭───────────────────┬───────────────────────────────────────────────────────────────────────╮
        │ Attribute         │ Value                                                                 │
        ├───────────────────┼───────────────────────────────────────────────────────────────────────┤
        │ X_train: str      │ datasets/healthcare/X_train_new.csv                                   │
        │ X_test: str       │ datasets/healthcare/X_test_new.csv                                    │
        │ y_train: str      │ datasets/healthcare/y_train_new.csv                                   │
        │ y_test: str       │ datasets/healthcare/y_test_new.csv                                    │
        │ description: dict │ {'NUM_SAMPLES': 1000, 'FEATURES': ['Age', 'BMI', ' ... } (length: 11) │
        ╰───────────────────┴───────────────────────────────────────────────────────────────────────╯

## Fast graph

In [42]:
from caia.fast.fast_graph import FastGraph
from caia.utils import save_yaml_results

working_memory = WorkingMemory(
    episodic_memory=init_episodic_memory,
    semantic_memory=init_semantic_memory,
    threshold=0.05,
    generations_fast_graph={},
    generations_slow_graph={},
    improvement_history=[],
)

fast_graph = FastGraph(llm_generator, debug=False)
output_fast_graph = fast_graph.run(working_memory)

short_uuid = str(uuid.uuid4())[:8]
filename = f"results/fast_temp_{TEMPERATURE}_max_iter_{MAX_ITERATIONS}_llm_{LLM_NAME.split('/')[1].split(':')[0]}_dataset_{dataset_folder.split('/')[-1]}_{short_uuid}.yaml"
save_yaml_results(output_fast_graph, filename)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                         Node: generate_retraining_code                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

No slow graph insights available, using basic retraining approach

╭─────────────────────────────────────────────── new_training_code ───────────────────────────────────────────────╮
│ new_training_code: |                                                                                            │
│   import yaml                                                                                                   │
│   import pandas as pd                                                                                           │
│   from sklearn.ensemble import RandomForestClassifier                                                           │
│                                                                                                                 │
│   # Initialize metrics dictionaries                                                                             │
│   model_new_score = {                                                                                           │
│       'on_new_data': 0.0,                                                                                       │
│       'on_old_data': 0.0                                                                                        │
│   }                                                                                                             │
│   model_old_score = {                                                                                           │
│       'on_new_data': 0.0,                                                                                       │
│       'on_old_data': 0.0                                                                                        │
│   }                                                                                                             │
│                                                                                                                 │
│   # load the old data                                                                                           │
│   dataset_folder = "datasets/healthcare"                                                                        │
│   X_train_old = pd.read_csv(f"{dataset_folder}/X_train_old.csv")                                                │
│   X_test_old = pd.read_csv(f"{dataset_folder}/X_test_old.csv")                                                  │
│   y_train_old = pd.read_csv(f"{dataset_folder}/y_train_old.csv").squeeze("columns")                             │
│   y_test_old = pd.read_csv(f"{dataset_folder}/y_test_old.csv").squeeze("columns")                               │
│                                                                                                                 │
│   model_old = RandomForestClassifier(random_state=42)                                                           │
│                                                                                                                 │
│   model_old.fit(X_train_old, y_train_old)                                                                       │
│                                                                                                                 │
│   # Test old model on old test set                                                                              │
│   old_score_old = model_old.score(X_test_old, y_test_old)                                                       │
│   print(f'Old model trained and evaluated on the old distribution: {old_score_old}')                            │
│   model_old_score['on_old_data'] = float(old_score_old)                                                         │
│                                                                                                                 │
│   # Test old model on new test set                                                                              │
│   X_test_new = pd.read_csv(f"{dataset_folder}/X_test_new.csv")                                                  │
│   y_test_new = pd.read_csv(f"{dataset_folder}/y_test_n

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                          Node: execute_retraining_code                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...


...

╭─────────────────────────────────────────────── execution_success ───────────────────────────────────────────────╮
│ True                                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── extracted_metrics ───────────────────────────────────────────────╮
│ {'model_old_score': {'on_old_data': 0.86, 'on_new_data': 0.7333333333333333}, 'model_new_score':                │
│ {'on_old_data': 0.87, 'on_new_data': 0.8333333333333334}}                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── iteration_count ────────────────────────────────────────────────╮
│ 1                                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── Latest Improvement ───────────────────────────────────────────────╮
│ Outcome: success                                                                                                │
│ Improvements:                                                                                                   │
│   New Distribution: 0.1000                                                                                      │
│   Old Distribution: 0.0100                                                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Results saved to: results/fast_temp_1.0_max_iter_1_llm_llama-3.1-8b-instruct_dataset_healthcare_2e1630bf.yaml

## Slow graph

In [43]:
from caia.slow.slow_graph import SlowGraph
from caia.utils import save_yaml_results

slow_graph = SlowGraph(llm_generator, debug=False)
working_memory["max_iterations"] = MAX_ITERATIONS
working_memory["max_failures"] = 5
output_slow_graph = slow_graph.run(working_memory)


short_uuid = str(uuid.uuid4())[:8]
filename = f"results/slow_temp_{TEMPERATURE}_max_iter_{MAX_ITERATIONS}_llm_{LLM_NAME.split('/')[1].split(':')[0]}_dataset_{dataset_folder.split('/')[-1]}_{short_uuid}.yaml"
save_yaml_results(output_slow_graph, filename)

Max iterations set to: 1

Max consecutive failures set to: 5

==================== STARTING ITERATION 1/1 ====================

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                         Node: check_fast_graph_results                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Detected Fast Graph Results from generations_fast_graph: --------------------------------------------------

...

╭────────────────────────────────────── Generation Update: iteration_count ───────────────────────────────────────╮
│ 2                                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── Latest Improvement ───────────────────────────────────────────────╮
│ Strategy: ensemble_method                                                                                       │
│ Outcome: success                                                                                                │
│ Improvements:                                                                                                   │
│   New Distribution: 0.1000                                                                                      │
│   Old Distribution: 0.0067                                                                                      │
│ Evaluation: reject                                                                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Strategy Progress ───────────────────────────────────────────────╮
│   [○] model_selection                                                                                           │
│   [✓] hyperparameter_tuning                                                                                     │
│ → [✓] ensemble_method                                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Iteration 1 time: 89.87 seconds

Results saved to: results/slow_temp_1.0_max_iter_1_llm_llama-3.1-8b-instruct_dataset_healthcare_b8db79e1.yaml

## Fast graph again

In [46]:
from caia.fast.fast_graph import FastGraph
from caia.utils import save_yaml_results

working_memory = WorkingMemory(
    episodic_memory=init_episodic_memory,
    semantic_memory=init_semantic_memory,
    threshold=0.05,
    generations_fast_graph={},
    generations_slow_graph=output_slow_graph,
    improvement_history=[],
)

fast_graph = FastGraph(llm_generator, debug=False)
output_fast_graph = fast_graph.run(working_memory)

short_uuid = str(uuid.uuid4())[:8]
print(short_uuid)
filename = f"results/improver_temp_{TEMPERATURE}_max_iter_{MAX_ITERATIONS}_llm_{LLM_NAME.split('/')[1].split(':')[0]}_dataset_{dataset_folder.split('/')[-1]}_{short_uuid}.yaml"
save_yaml_results(output_fast_graph, filename)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                         Node: generate_retraining_code                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Using insights from slow graph to enhance retraining code generation

╭─────────────────────────────────────────────── new_training_code ───────────────────────────────────────────────╮
│ new_training_code: |                                                                                            │
│   import yaml                                                                                                   │
│   import pandas as pd                                                                                           │
│   from sklearn.ensemble import RandomForestClassifier, VotingClassifier                                         │
│   from sklearn.metrics import accuracy_score                                                                    │
│                                                                                                                 │
│   # Initialize metrics dictionary                                                                               │
│   model_new_score = {                                                                                           │
│       'on_new_data': 0.0,                                                                                       │
│       'on_old_data': 0.0                                                                                        │
│   }                                                                                                             │
│                                                                                                                 │
│   # Load data from specified folder                                                                             │
│   dataset_folder = "datasets/healthcare"                                                                        │
│   X_train_old = pd.read_csv(f"{dataset_folder}/X_train_old.csv")                                                │
│   y_train_old = pd.read_csv(f"{dataset_folder}/y_train_old.csv").squeeze("columns")                             │
│   X_test_old = pd.read_csv(f"{dataset_folder}/X_test_old.csv")                                                  │
│   y_test_old = pd.read_csv(f"{dataset_folder}/y_test_old.csv").squeeze("columns")                               │
│                                                                                                                 │
│   X_train_new = pd.read_csv(f"{dataset_folder}/X_train_new.csv")                                                │
│   y_train_new = pd.read_csv(f"{dataset_folder}/y_train_new.csv").squeeze("columns")                             │
│   X_test_new = pd.read_csv(f"{dataset_folder}/X_test_new.csv")                                                  │
│   y_test_new = pd.read_csv(f"{dataset_folder}/y_test_new.csv").squeeze("columns")                               │
│                                                                                                                 │
│                                                                                                                 │
│   try:                                                                                                          │
│       # Train model with optimized parameters on old data and evaluate                                          │
│       model_old = RandomForestClassifier(n_estimators=200)                                                      │
│       model_old.fit(X_train_old, y_train_old)                                                                   │
│                                                                                                                 │
│       # Test model on old distribution                                                                          │
│       old_score_old = model_old.score(X_test_old, y_test_old)                                                   │
│       print(f'Old model trained and evaluated on the old distribution: {old_score_old}')                        │
│       model_new_score['on_old_data'] = float(old_score

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                          Node: execute_retraining_code                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...


...

╭─────────────────────────────────────────────── extracted_metrics ───────────────────────────────────────────────╮
│ {'model_old_score': {'on_old_data': 0.8566666666666667, 'on_new_data': 0.7}, 'model_new_score': {'on_old_data': │
│ 0.8766666666666667, 'on_new_data': 0.8333333333333334}}                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── iteration_count ────────────────────────────────────────────────╮
│ 1                                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── Latest Improvement ───────────────────────────────────────────────╮
│ Outcome: success                                                                                                │
│ Improvements:                                                                                                   │
│   New Distribution: 0.1333                                                                                      │
│   Old Distribution: 0.0200                                                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

8be3ad20

Results saved to: results/improver_temp_1.0_max_iter_1_llm_llama-3.1-8b-instruct_dataset_healthcare_8be3ad20.yaml